# Axe 2

#### Packages

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import random


### 1) Processing du df composant médical - Emission carbonne (kgCO2e)

In [ ]:


def concat_xlsx_from_folder(folder_path: str) -> pd.DataFrame:
    """
    Parcourt récursivement un dossier git et concatène verticalement
    toutes les tables issues des fichiers .xlsx.

    Hypothèses :
    - chaque fichier .xlsx contient une table avec exactement 2 colonnes
    - la première ligne correspond aux labels et est ignorée
    - colonne A : produit
    - colonne B : Emission_kgCO2e_unitaire
    """

    folder = Path(folder_path)
    all_rows = []

    for file in folder.rglob("*.xlsx"):
        df = pd.read_excel(file, header=0)

        df = df.iloc[:, :2]
        df.columns = ["produit", "Emission_kgCO2e_unitaire"]

        filename = file.stem
        if filename.endswith("_parProduit"):
            df["type_de_donnees"] = "parProduit"
        elif filename.endswith("_m2"):
            df["type_de_donnees"] = "m2"
        elif filename.endswith("_parKG"):
            df["type_de_donnees"] = "parKG"
        else:
            continue

        all_rows.append(df)

    if not all_rows:
        return pd.DataFrame(
            columns=["produit", "Emission_kgCO2e_unitaire", "type_de_donnees"]
        )

    df = pd.concat(all_rows, axis=0, ignore_index=True)

    df["Emission_kgCO2e_unitaire"] = (
        df["Emission_kgCO2e_unitaire"]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    
    df["Emission_kgCO2e_unitaire"] = pd.to_numeric(
        df["Emission_kgCO2e_unitaire"],
        errors="coerce"
    )
    
    return df


def concat_xlsx_from_folders(list_paths: list[str | Path]) -> pd.DataFrame:
    """
    Concatène verticalement les tables issues de plusieurs dossiers.

    Paramètre
    ----------
    list_paths : list[str | Path]
        Liste de chemins vers des dossiers contenant des fichiers .xlsx

    Retour
    ------
    DataFrame avec les colonnes :
    - produit
    - Emission_kgCO2e_unitaire
    - type_de_donnees
    """

    all_rows = []

    for folder_path in list_paths:
        folder = Path(folder_path)

        if not folder.exists():
            raise FileNotFoundError(f"Dossier introuvable : {folder}")

        xlsx_files = list(folder.rglob("*.xlsx"))
        if not xlsx_files:
            raise ValueError(f"Aucun fichier .xlsx trouvé dans {folder}")

        for file in xlsx_files:
            df = pd.read_excel(file)

            df = df.iloc[:, :2]
            df.columns = ["produit", "Emission_kgCO2e_unitaire"]

            name = file.stem
            if name.endswith("_parProduit"):
                df["type_de_donnees"] = "parProduit"
            elif name.endswith("_m2"):
                df["type_de_donnees"] = "m2"
            elif name.endswith("_parKG"):
                df["type_de_donnees"] = "parKG"
            else:
                continue

            all_rows.append(df)

    if not all_rows:
        return pd.DataFrame(
            columns=["produit", "Emission_kgCO2e_unitaire", "type_de_donnees"]
        )

    df = pd.concat(all_rows, axis=0, ignore_index=True)
    
    df["Emission_kgCO2e_unitaire"] = (
        df["Emission_kgCO2e_unitaire"]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )

    df["Emission_kgCO2e_unitaire"] = pd.to_numeric(
        df["Emission_kgCO2e_unitaire"],
        errors="coerce"
    )
    
    return df



def random_value_dict(
    df: pd.DataFrame,
    nom_col: str = "produit",
    nom_to_ignore: list | None = None,
    min_value: float = 1.0,
    max_value: float = 3000.0,
) -> dict:
    """
    Construit un dictionnaire :
    - clés : valeurs uniques de df[nom_col]
    - valeurs : nombre aléatoire strictement > 1
      tiré dans [min_value, max_value]
    """

    if nom_to_ignore is None:
        nom_to_ignore = []

    if min_value <= 1:
        min_value = 1.000001

    uniques = df[nom_col].dropna().unique()

    return {
        val: random.uniform(min_value, max_value)
        for val in uniques
        if val not in nom_to_ignore
    }


def compute_emission_hopital(
    df: pd.DataFrame,
    type_de_donnees: str = "type_de_donnees",
    dict_m2: dict = {"pansements composites": (3 * 10**2, 10)},
    dict_parKG: dict = {"instrument usage unique": 1000, "complement alimentaire":10000},
    dict_nb_parProduit: dict | None = None,
) -> pd.DataFrame:
    """
    Ajoute la colonne Emission_carbonne_total_des_produits_kgCO2e selon
    le type de données associé à chaque produit.
    """

    if dict_nb_parProduit is None:
        dict_nb_parProduit = {}

    def compute_row(row):
        produit = row["produit"]
        emission_unit = row["Emission_kgCO2e_unitaire"]
        t = row[type_de_donnees]

        if t == "parProduit":
            return emission_unit * dict_nb_parProduit.get(produit, np.nan)

        if t == "m2":
            longueur, largeur = dict_m2.get(produit, (np.nan, np.nan))
            return emission_unit * longueur * largeur

        if t == "parKG":
            return emission_unit * dict_parKG.get(produit, np.nan)

        return np.nan

    df = df.copy()
    df["Emission_carbonne_total_des_produits_kgCO2e"] = df.apply(compute_row, axis=1)

    return df

# =========================
# 3. Construction du DataFrame
# =========================

# extract_path = "sujets/chu/Axe_2/Axe_2_bdd"
# extract_path = "C:/Users/jerem/Documents/GitHub/datachallenge2026/sujets/chu/Axe_2/Axe_2_bdd"
extract_path = "/home/onyxia/datachallenge2026/sujets/chu/Axe_2/Axe_2_bdd"
# paths = [
#     r"sujets\chu\Axe_2\Axe_2_bdd-20260117T004817Z-1-001\Axe_2_bdd",
#     r"sujets\chu\Axe_2\autre_dossier"
# ]

df_concat = concat_xlsx_from_folder(extract_path)
# df_concat = concat_xlsx_from_folders(paths)

print(df_concat.head())

# =========================
# 4. Dictionnaires exemples
# =========================

dict_nb_parProduit = random_value_dict(df_concat)

print(dict_nb_parProduit)

# =========================
# 5. Calcul des émissions
# =========================

df_final = compute_emission_hopital(
    df_concat,
    dict_nb_parProduit=dict_nb_parProduit
)

print(df_final.head())



                   produit  Emission_kgCO2e_unitaire type_de_donnees
0    pansements composites                    0.9400              m2
1           Sonde urinaire                   51.9034      parProduit
2  Set de sondage urinaire                   89.4064      parProduit
3      Collecteur de jambe                  250.7186      parProduit
4       Collecteur de nuit                  676.6522      parProduit
{'pansements composites': 2880.7506438973483, 'Sonde urinaire': 2.3058391190603773, 'Set de sondage urinaire': 1301.0278534529064, 'Collecteur de jambe': 1871.2095479103848, 'Collecteur de nuit': 2265.768898794339, 'Etuis péniens': 1088.7167868279912, 'Poche pour stomie': 3.131259971503482, 'Support pour stomie': 2959.0238083770532, 'Changes complets': 2375.889528275952, 'Slips absorbants': 250.72891706628525, 'Protections absorbantes': 1887.063988630245, 'Couches droites': 1329.456550312355, 'Alèses': 1383.9846297425656, 'pansement': 2495.056711895948, 'uteruscopes': 2579.211406

In [35]:
# Doublons sur une variable particulière (ex: 'email')
colonne = 'produit'

# Trouver les valeurs dupliquées dans cette colonne
valeurs_doublons = df_final[df_final.duplicated(subset=[colonne], keep=False)]

# Afficher les doublons triés pour mieux voir
doublons_tries = valeurs_doublons.sort_values(colonne)
print(f"Doublons sur la colonne '{colonne}' :")
print(doublons_tries)

# Voir les valeurs qui se répètent
comptage = df_final[colonne].value_counts()
valeurs_repetees = comptage[comptage > 1]
print(f"\nValeurs répétées dans '{colonne}' :")
print(valeurs_repetees)


Doublons sur la colonne 'produit' :
Empty DataFrame
Columns: [produit, Emission_kgCO2e_unitaire, type_de_donnees, Emission_carbonne_total_des_produits_kgCO2e]
Index: []

Valeurs répétées dans 'produit' :
Series([], Name: count, dtype: int64)


#### 1.1) Obtention du dataframe final et exportation

In [36]:
colonnes_a_conserver = [
    "produit",
    "Emission_kgCO2e_unitaire",
    "Emission_carbonne_total_des_produits_kgCO2e"
]

df_export = df_final[colonnes_a_conserver].copy()

# =========================
# Chemin de sortie
# =========================

output_path = Path(r"results\df_composant_medical_emissions_carbones.xlsx")

# Création du dossier si besoin
output_path.parent.mkdir(parents=True, exist_ok=True)

# =========================
# Export Excel
# =========================

df_export.to_excel(output_path, index=False)

### 2) Pre processing NLP des bases **df_composant_medical_emissions_carbones.xlsx** et **DISPOSITIFS_MED.xlsx** et Classification

In [1]:
# on sup un encoding: utf-8
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

!pip install transformers
from transformers import pipeline

!pip install rank_bm25
from rank_bm25 import BM25Okapi

!pip install spacy transformers sentencepiece torch
!python -m spacy download fr_core_news_md
!python -m spacy download en_core_web_sm
!pip install rank-bm25
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 72.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 66.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 87.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 75.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 92.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 100.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [sentence_transformers]ence_transformers]


/opt/python/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 91.3 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 835.9/835.9 kB 78.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 99.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 74.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 91.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22/22 [spacy]m21/22 [spacy]ic]core]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 85.5 MB/s  0:00:00 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.0 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:

# ============================================================
# 0) I/O + sélection colonnes df2
# ============================================================

DF2_KEEP_COLS = [
    "Nomenclature achat",
    "Catégories d'achat\n(N-2)",
    "Segments  d'achat\n(N-3)",
    "Sous-segment",
    "Produit élémentaire",
    "Code des Catégories Homogènes \nde fournitures et prestations",
]

def load_and_select_df2(path_df2_xlsx: str) -> pd.DataFrame:
    df2 = pd.read_excel(path_df2_xlsx)
    missing = [c for c in DF2_KEEP_COLS if c not in df2.columns]
    if missing:
        raise ValueError(f"Colonnes manquantes dans df2: {missing}\nColonnes trouvées: {list(df2.columns)}")
    return df2[DF2_KEEP_COLS].copy()




# ============================================================
# 1) Prétraitement + traduction (identique logique)
# ============================================================



def build_fr_nlp(model_name: str = "fr_core_news_md"):
    import spacy
    return spacy.load(model_name, disable=["ner", "parser"])

def build_en_nlp(model_name: str = "en_core_web_sm"):
    import spacy
    return spacy.load(model_name, disable=["ner", "parser"])

def preprocess_fr(texts: Iterable[str], nlp=None) -> List[str]:
    if nlp is None:
        nlp = build_fr_nlp()
    out = []
    for doc in nlp.pipe([("" if x is None else str(x)) for x in texts], batch_size=256):
        toks = []
        for t in doc:
            if t.is_space or t.is_punct or t.like_num:
                continue
            if t.is_stop:
                continue
            lem = (t.lemma_ or t.text).lower().strip()
            if len(lem) < 2:
                continue
            toks.append(lem)
        out.append(" ".join(toks))
    return out

def preprocess_en(texts: Iterable[str], nlp=None) -> List[str]:
    if nlp is None:
        nlp = build_en_nlp()
    out = []
    for doc in nlp.pipe([("" if x is None else str(x)) for x in texts], batch_size=256):
        toks = []
        for t in doc:
            if t.is_space or t.is_punct or t.like_num:
                continue
            if t.is_stop:
                continue
            lem = (t.lemma_ or t.text).lower().strip()
            if len(lem) < 2:
                continue
            toks.append(lem)
        out.append(" ".join(toks))
    return out

@dataclass
class TranslatorFR2EN:
    model_name: str = "Helsinki-NLP/opus-mt-fr-en"
    device: int = -1  # -1 CPU, 0 GPU

    def __post_init__(self):
        self.pipe = pipeline("translation", model=self.model_name, device=self.device)

    def translate(self, texts: Iterable[str], batch_size: int = 16) -> List[str]:
        texts_list = [("" if x is None else str(x)) for x in texts]
        outputs = self.pipe(texts_list, batch_size=batch_size, truncation=True)
        return [o["translation_text"] for o in outputs]


def add_processed_columns(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    col_df1_produit: str = "produit",
    col_df2_best: str = "Produit élémentaire",
    translator: Optional[TranslatorFR2EN] = None,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    On se concentre sur 'Produit élémentaire' pour df2 (champ le plus proche),
    et on garde aussi un champ df2 '__df2_join_en_proc' pour éventuellement enrichir.
    """
    if col_df1_produit not in df1.columns:
        raise ValueError(f"df1 n'a pas la colonne {col_df1_produit}")
    if col_df2_best not in df2.columns:
        raise ValueError(f"df2 n'a pas la colonne {col_df2_best}")

    fr_nlp = build_fr_nlp()
    en_nlp = build_en_nlp()
    if translator is None:
        translator = TranslatorFR2EN()

    df1 = df1.copy()
    df2 = df2.copy()

    # df1 produit
    df1["produit_fr_proc"] = preprocess_fr(df1[col_df1_produit].astype(str), nlp=fr_nlp)
    df1["produit_en"] = translator.translate(df1["produit_fr_proc"].tolist())
    df1["produit_en_proc"] = preprocess_en(df1["produit_en"], nlp=en_nlp)

    # df2 produit élémentaire (principal)
    df2["produit_elem_fr_proc"] = preprocess_fr(df2[col_df2_best].astype(str), nlp=fr_nlp)
    df2["produit_elem_en"] = translator.translate(df2["produit_elem_fr_proc"].tolist())
    df2["produit_elem_en_proc"] = preprocess_en(df2["produit_elem_en"], nlp=en_nlp)

    # champ joint optionnel (pondération: Produit élémentaire x3)
    # utile si tu veux plus tard intégrer d'autres colonnes, sans casser l'approche
    df2["__df2_join_en_proc"] = (
        (df2["produit_elem_en_proc"].fillna("") + " ") * 3
    ).str.replace(r"\s+", " ", regex=True).str.strip()

    return df1, df2


# ============================================================
# 2) Filtre lexical BM25 (avant embeddings)
# ============================================================

def bm25_candidates(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    df1_text_col: str = "produit_en_proc",
    df2_text_col: str = "produit_elem_en_proc",
    topk_bm25: int = 20,
) -> np.ndarray:
    """
    Retourne un tableau d'indices (n_df2, topk_bm25) : les meilleurs candidats df1
    pour chaque ligne df2 selon BM25.

    On tokenize simplement par split() car les textes sont déjà normalisés.
    """
    

    corpus_tokens = [str(x).split() for x in df1[df1_text_col].fillna("").tolist()]
    bm25 = BM25Okapi(corpus_tokens)

    cand_idx = np.zeros((df2.shape[0], topk_bm25), dtype=int)

    for i, q in enumerate(df2[df2_text_col].fillna("").tolist()):
        q_tokens = str(q).split()
        scores = bm25.get_scores(q_tokens)  # (n_df1,)
        best = np.argsort(-scores)[:topk_bm25]
        cand_idx[i, :] = best

    return cand_idx


# ============================================================
# 3) Rerank embeddings sur candidats + proba Top-5
# ============================================================

def embed_texts(texts: List[str], model_name: str = "pritamdeka/S-PubMedBert-MS-MARCO") -> np.ndarray:
    model = SentenceTransformer(model_name)
    emb = model.encode(texts, normalize_embeddings=True, batch_size=64, show_progress_bar=True)
    return np.asarray(emb)

def softmax(x: np.ndarray, temperature: float = 0.07) -> np.ndarray:
    x = x / max(temperature, 1e-6)
    x = x - x.max(axis=1, keepdims=True)
    expx = np.exp(x)
    return expx / expx.sum(axis=1, keepdims=True)

def match_with_bm25_then_embeddings(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    col_df1_key: str = "produit",
    df1_text_col: str = "produit_en_proc",
    df2_text_col: str = "produit_elem_en_proc",
    topk_bm25: int = 20,
    topk_final: int = 5,
    embedding_model: str = "pritamdeka/S-PubMedBert-MS-MARCO",
    temperature: float = 0.07,
) -> pd.DataFrame:
    """
    Pipeline:
    - BM25 filtre les candidats df1 (topk_bm25)
    - embeddings rerank uniquement ces candidats
    - softmax sur similarités => pseudo-proba
    - renvoie un tableau wide top-5 (et long via attrs)
    """

    # df1 unique
    df1u = df1[[col_df1_key, df1_text_col]].drop_duplicates(subset=[col_df1_key]).reset_index(drop=True)

    # candidats BM25
    cand_idx = bm25_candidates(
        df1u,
        df2,
        df1_text_col=df1_text_col,
        df2_text_col=df2_text_col,
        topk_bm25=topk_bm25,
    )  # (n2, topk_bm25)

    # embeddings df1 (une seule fois)
    emb1 = embed_texts(df1u[df1_text_col].fillna("").tolist(), model_name=embedding_model)

    # embeddings df2 (sur champ principal)
    emb2 = embed_texts(df2[df2_text_col].fillna("").tolist(), model_name=embedding_model)

    # calcul similarities restreint
    n2 = df2.shape[0]
    sims = np.empty((n2, topk_bm25), dtype=float)

    for i in range(n2):
        idx = cand_idx[i]
        sims[i, :] = emb2[i] @ emb1[idx].T  # cosine car normalisé

    probs = softmax(sims, temperature=temperature)  # (n2, topk_bm25)

    # topk_final parmi candidats
    top_local = np.argsort(-probs, axis=1)[:, :topk_final]             # indices 0..topk_bm25-1
    top_prob = np.take_along_axis(probs, top_local, axis=1)            # (n2, topk_final)
    top_global_idx = np.take_along_axis(cand_idx, top_local, axis=1)   # indices dans df1u
    top_prod = df1u[col_df1_key].to_numpy()[top_global_idx]            # (n2, topk_final)

    # outputs
    rows = []
    for i in range(n2):
        for r in range(topk_final):
            rows.append({
                "Nomenclature achat": df2.iloc[i]["Nomenclature achat"],
                "rank": r + 1,
                "produit_match": top_prod[i, r],
                "proba": float(top_prob[i, r]),
            })
    out_long = pd.DataFrame(rows)

    wide = {"Nomenclature achat": df2["Nomenclature achat"].to_numpy()}
    for r in range(topk_final):
        wide[f"top{r+1}_produit"] = top_prod[:, r]
        wide[f"top{r+1}_proba"] = top_prob[:, r]
    out_wide = pd.DataFrame(wide)

    out_wide.attrs["out_long"] = out_long
    return out_wide


# ============================================================
# 4) Utilitaires pratiques
# ============================================================

def keep_df2_columns(df2: pd.DataFrame) -> pd.DataFrame:
    # mêmes colonnes que tu veux conserver
    keep = [
        "Nomenclature achat",
        "Catégories d'achat\n(N-2)",
        "Segments  d'achat\n(N-3)",
        "Sous-segment",
        "Produit élémentaire",
        "Code des Catégories Homogènes \nde fournitures et prestations",
    ]
    missing = [c for c in keep if c not in df2.columns]
    if missing:
        raise ValueError(f"Colonnes manquantes: {missing}\nColonnes df2: {list(df2.columns)}")
    return df2[keep].copy()


# ============================================================
# 5) Exemple d'exécution
# ============================================================

if __name__ == "__main__":
    # path_df1 = r"df_composant_medical_emissions_carbones.xlsx"
    # path_df2 = r"DISPOSITIFS_MED.xlsx"

    path_df1 = r"/home/onyxia/datachallenge2026/sujets/chu/Axe_2/results/df_composant_medical_emissions_carbones.xlsx"
    path_df2 = r"/home/onyxia/datachallenge2026/sujets/chu/Axe_2/DISPOSITIFS_MED.xlsx"

    df1 = pd.read_excel(path_df1)
    df2 = load_and_select_df2(path_df2)

    # NLP + traduction
    translator = TranslatorFR2EN(device=-1)  # CPU
    df1p, df2p = add_processed_columns(
        df1, df2,
        col_df1_produit="produit",
        col_df2_best="Produit élémentaire",
        translator=translator
    )

    # Matching BM25 -> Embeddings -> Top5
    match_wide = match_with_bm25_then_embeddings(
        df1p, df2p,
        col_df1_key="produit",
        df1_text_col="produit_en_proc",
        df2_text_col="produit_elem_en_proc",
        topk_bm25=20,         
        topk_final=5,
        embedding_model="pritamdeka/S-PubMedBert-MS-MARCO",
        temperature=0.07
    )

    # Sauvegarde
    match_wide.to_excel("MATCH_df2_vers_df1_top5.xlsx", index=False)
    match_wide.attrs["out_long"].to_excel("MATCH_df2_vers_df1_top5_long.xlsx", index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'df_composant_medical_emissions_carbones.xlsx'

### 3) Réduction et évaluation des coûts carbones 

In [ ]:
# SLM, TintBERT ?
# Métriques et graphes de consommation carbonne par inférence
